In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import sent2vec, cos_sim
import re

In [ ]:
pairs = pd.read_csv(diffusion_graph_dir+'article_pairs.tsv', sep='\t')
article_details = pd.read_csv('cache/article_details_v2.tsv', sep='\t')
paper_details = pd.read_csv('cache/paper_details_v2.tsv', sep='\t')

In [ ]:
pairs = pairs.merge(paper_details[['url','paragraphs', 'full_text']], left_on='paper', right_on='url').merge(article_details[['url','paragraphs', 'full_text']], left_on='article', right_on='url')[['paper', 'paragraphs_x', 'full_text_x', 'article', 'paragraphs_y', 'full_text_y']]

In [ ]:
def compute_similarity(pairs):
    
    paragraphs_x = eval(pairs['paragraphs_x'])
    paragraphs_y = eval(pairs['paragraphs_y'])
    full_text_x = re.split('\n', pairs['full_text_x'])
    full_text_y = re.split('\n', pairs['full_text_x'])
    
    sum_sim = 0.0
    for px, ftx in zip(paragraphs_x, full_text_x):
        px = set(px)
        ftxv = sent2vec(ftx)
        max_sim = 0.0
        pair = ''
        for py, fty in zip(paragraphs_y, full_text_y):
            ftyv = sent2vec(fty)
            sim = min((1 + (len(px.intersection(py)) / (len(px.union(py))+1))) * cos_sim(ftxv, ftyv) * (min(len(ftx), len(fty)) / (max(len(ftx), len(fty)) +1)), 1.0)
            if sim > max_sim:
                max_sim = sim
                pair = 'Pair:' + str(ftx) + '--' + str(fty) + '--' + str(sim) + '--' + str((len(px.intersection(py)) / (len(px.union(py))+1))) + '--' + str(cos_sim(ftxv, ftyv))
                
        if max_sim > 0.8:
            print(pair)
        sum_sim += max_sim
        
    return sum_sim / len(paragraphs_x)


print(pairs.iloc[2343]['paper'], pairs.iloc[2343]['article'])
compute_similarity(pairs.iloc[5993])